## Random Forest

Random Forest is one of my favorite methods because it is pretty easy to use and returns pretty good results. The data has already been cleaned up so I can get straight to the modeling from here. I'll also be creating a table of results I get from the algorithm. The first table will be based on performance and accuracy I'll be manipulating the data and my settings to get the best result.

Here is some data from earlier runs that I did earlier in the week. I don't have enough RAM on my computer to run a proper Random Forest on the full data set so I tried changing some of the settings to see what type of results I would get, so far it things don't look so great.

| Out of bag Score               | Depth | Trees |
|---------------------|-------|-------|
| 0.24406610565014025 | 7     | 250   |
| 0.243709633517      | 7     | 500   |
| 0.244111661194      | 7     | 750   |
| 0.243918050132      | 7     | 1000  |
| 0.29118306609312239 | 15    | 250   |

In [5]:
import pandas as pd
import numpy as np
Y = pd.read_csv('./data/Y.csv', index_col = 0, header=None, names=['Crime'])
X = pd.read_csv('./data/X.csv', index_col=0)

In [2]:
rows = np.random.choice(X.index.values, X.shape[0])
a = X.ix[rows]
b = Y['Crime'].ix[rows]

In [3]:
import sklearn.ensemble as sk
n = 25
rfc = sk.RandomForestClassifier(n_estimators = n, oob_score = True, bootstrap = True)

In [43]:
# 1% 80mb 453ms 18.34% 25
# 1% 750mb 3.52ms 22.79% 250
# 1% 2500
# 1% 2800mb 13.4s 24.27% 1000
# 10% 700mb 4.07s 27.91% 25
# 30% 1700mb 13.5s 40.68% 25
# 30% 100
# 30% 4000mb 25.5s 42.39% 50
# 50% 31000mb 24.7s 49.38% 25
# 70% 4200mb  38.7s 55.92% 25

In [4]:
%time rfc.fit(a,b)
rfc.oob_score_

Wall time: 1min 53s


C:\Users\2015\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


0.63114814776851857

On my 2nd try with Random Forest I focused more on my different parameters than anything else. After experimenting with different settings I was able too get some improvements from my past scores but I still have a ways to go.

| Sample Size | Memory (RAM) | Time  | oob Score | Estimators | Comments and Notes                                                           |
|-------------|--------------|-------|-----------|------------|------------------------------------------------------------------------------|
| 1%          |              |       |           | 2500       | Memory went over 5000Mb process was ended                                    |
| 1%          | 2800Mb       | 13.4s | 24.27%    | 1000       |                                                                              |
| 10%         | 1700Mb       | 4.07s | 27.91%    | 25         |                                                                              |
| 30%         | 4000Mb       | 25.5s | 42.39%    | 50         |                                                                              |
| 50%         | 3100Mb       | 24.7s | 49.28%    | 25         | Given warning that some inputs did not have OOB scores, due to too few trees |
| 70%         | 4200Mb       | 38.7s | 55.92%    | 25         | Given warning that some inputs did not have OOB scores, due to too few trees |

In [18]:
df = pd.read_csv('data/train.csv')
Z = df.drop(['Category','Descript','Resolution','X','Y'],1)
n = 0
for col in Z.columns:
    if n < 4:
        Z[col] = Z[col].astype('category')
        Z[col] = Z[col].cat.rename_categories(range(0,Z[col].nunique()))
    n += 1

In [37]:
%time rfc.fit(Z.ix[rows],b)
rfc.oob_score_

Wall time: 1min 4s


0.61514291788359043

In [33]:
#70% 4400Mb N/A 52.89% 25 all but X,Y
#50% 50 all but X,Y
#50% 25 all but X,Y,Day
#50% 1300Mb 20.2s 30% 25 DayofWeek, Address
#80% 1600Mb 34.2s 33.17% 25 DayofWeek, Address
#80%  5200Mb 5min 30s 33.41% 100 DayofWeek, Address
#80% 800Mb  32.6s 30.89% 25 PdDistrict, Address, X, Y
#80% 2000Mb  35.7s 36.72% 25 All but day Month year
#100% 2300Mb  46s 37.88% 25 All but day Month year
#70% 4400Mb 52.6s 56.80% 25 original data
#70% 4400Mb 50.5s 56.73% 25 original data without XY
#80% 5000Mb 55s 58.79% 25 original data without XY
#90% 5000Mb 55s 61.15% 25 original data without XY

Slimming Down my data back to the original features really improved the performance and accuracy of my models. By Dropping the X,Y columns I was able to get the oob_score up to 61%

| Sample Size | Memory | time  | Oob Score | Estimators | Data Columns               |
|-------------|--------|-------|-----------|------------|----------------------------|
| 70%         | 4400Mb | N/A   | 52.89%    | 25         | all but X,Y                |
| 100%        | 2300Mb | 46s   | 37.88%    | 25         | All but Day Month and Year |
| 80%         | 800Mb  | 32.6s | 30.89%    | 25         | PdDistrict, Address, X, Y  |
| 70%         | 4400Mb | 52.6s | 56.80%    | 25         | Original Data              |
| 80%         | 5000Mb | 55s   | 58.79%    | 25         | Original Data w/o X,Y      |
| 90%         | 5000Mb | 55s   | 61.15%    | 25         | Original Data w/o X,Y      |
| 100%         | 5000Mb | 1min 53s   | 63.11%    | 25         | Original Data w/o X,Y      |

In [2]:
from sklearn.externals import joblib
# joblib.dump(rfc, './models/rf/61rfc.pkl')

In [3]:
rfc = joblib.load('./models/rf/61rfc.pkl')

In [10]:
rfc.score(Z,Y)

0.62944095375087272

In [12]:
test = pd.read_csv('./data/test.csv', index_col=0)
test.drop(['X','Y'],1,inplace=True)
n = 0
for col in test.columns:
    if n < 4:
        test[col] = test[col].astype('category')
        test[col] = test[col].cat.rename_categories(range(0,test[col].nunique()))
    n += 1

In [24]:
%time results = rfc.predict(test)

Wall time: 4h 32min 42s


Wall time: 4h 32min 42s
rfc.score = .629
Kaggle Score: 33.75118

In [27]:
resultdf = pd.DataFrame(results)
resultdf.to_csv('rf_results.csv')

In [47]:
# categories = df.Category.unique()
for i,cat in enumerate(categories):
    resultdf.replace(i,cat,inplace=True)
resultdf.to_csv('rf_resultsformat.csv')

Turns out the submission wants the probabilities of each crime so I should've used the probablity predict probability method

In [13]:
%time rfc.predict_proba(test)

Wall time: 1h 44min 7s


array([[ 0.   ,  0.08 ,  0.   , ...,  0.16 ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.12 ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.04 ,  0.   ,  0.   ],
       ..., 
       [ 0.   ,  0.12 ,  0.08 , ...,  0.04 ,  0.04 ,  0.   ],
       [ 0.   ,  0.01 ,  0.   , ...,  0.   ,  0.01 ,  0.   ],
       [ 0.   ,  0.044,  0.   , ...,  0.   ,  0.04 ,  0.   ]])

In [14]:
probablity_results = _oh[13]

In [48]:
pr = pd.DataFrame(probablity_results, columns=df.Category.unique())

In [49]:
pr = pr[sorted(pr.columns.values)]
pr.insert(0,'Id',pr.index.values)

In [50]:
pr.to_csv('rf_probresults.csv', index=False)

This improved score by a little bit and ran a lot faster

Time: 1h 44min 7s Score: 28.49762